In [254]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# csv 파일 불러오기
file_path = '/home/u1043/machinelearning/practice/datasets_30069_39285_AppleStore.csv'
store_data = pd.read_csv(file_path)

store_data

# prime_genre = preprocessing.LabelEncoder()
# final_data = store_data.copy()

# final_data['prime_genre'] = prime_genre.fit_transform(final_data['prime_genre'])

# final_data

,Unnamed: 0,id,track_name,size_bytes,currency,price,rating_count_tot,rating_count_ver,user_rating,user_rating_ver,ver,cont_rating,prime_genre,sup_devices.num,ipadSc_urls.num,lang.num,vpp_lic
0,1,281656475,PAC-MAN Premium,100788224,USD,3.99,21292,26,4.0,4.5,6.3.5,4+,Games,38,5,10,1
1,2,281796108,Evernote - stay organized,158578688,USD,0.00,161065,26,4.0,3.5,8.2.2,4+,Productivity,37,5,23,1
2,3,281940292,"WeatherBug - Local Weather, Radar, Maps, Alerts",100524032,USD,0.00,188583,2822,3.5,4.5,5.0.0,4+,Weather,37,5,3,1
3,4,282614216,"eBay: Best App to Buy, Sell, Save! Online Shop...",128512000,USD,0.00,262241,649,4.0,4.5,5.10.0,12+,Shopping,37,5,9,1
4,5,282935706,Bible,92774400,USD,0.00,985920,5320,4.5,5.0,7.5.1,4+,Reference,37,5,45,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7192,11081,1187617475,Kubik,126644224,USD,0.00,142,75,4.5,4.5,1.3,4+,Games,38,5,1,1
7193,11082,1187682390,VR Roller-Coaster,120760320,USD,0.00,30,30,4.5,4.5,0.9,4+,Games,38,0,1,1
7194,11087,1187779532,Bret Michaels Emojis + Lyric Keyboard,111322112,USD,1.99,15,0,4.5,0.0,1.0.2,9+,Utilities,37,1,1,1
7195,11089,1187838770,VR Roller Coaster World - Virtual Reality,97235968,USD,0.00,85,32,4.5,4.5,1.0.15,12+,Games,38,0,2,1


In [4]:
from sklearn.model_selection import train_test_split, KFold
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
import numpy as np
import pandas as pd

def MAE(test_y, pred_y):
    return np.mean(np.abs((test_y - pred_y)))

# csv 파일 불러오기
file_path = '/home/u1043/machinelearning/practice/datasets_30069_39285_AppleStore.csv'
store_data = pd.read_csv(file_path)

# 데이터프레임 정보 출력
# store_data.info()

# final_data2.isnull().sum() # 결측 데이터가 있나 확인, 결측 데이터 없음

# 데이터 전처리 
# 필요 없는 feature Drop
del(store_data['Unnamed: 0']) # index 행
del(store_data['id']) # id값
del(store_data['track_name']) # 어플 이름
del(store_data['sup_devices.num']) # 지원 장치 수
del(store_data['currency']) # 통화 종류
del(store_data['vpp_lic']) # Vpp 장치 기반 라이센스 활성화 

# ver열의 데이터들의 .을 기준으로 첫번째 숫자만 저장하기
ver_split = store_data["ver"].str.split(".")
store_data["ver"] = ver_split.str.get(0)

# ver 열의 값들 중 문자열이 있는 행은 삭제
contains_string_df = store_data['ver'].str.contains('iOV 2|v2|v3|1,2|V3|v1|Update 1')
store_data = store_data[~contains_string_df]

# 범주형 데이터는 레이블인코딩
prime_genre = preprocessing.LabelEncoder()
final_data1 = store_data.copy()
final_data1['prime_genre'] = prime_genre.fit_transform(final_data1['prime_genre'])

cont_rating = preprocessing.LabelEncoder()
final_data2 = final_data1.copy()
final_data2['cont_rating'] = prime_genre.fit_transform(final_data2['cont_rating'])

# 전처리 후 데이터프레임 정보 출력
#final_data2.info()

# 사용할 feature들 변수에 담아주기
features = ['size_bytes', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'ipadSc_urls.num', 'lang.num']

"""[3]
* K- fold 교차 검증 사용
"""
kf = KFold(n_splits=10, shuffle=True) # 데이터를 10개로 나누기, suffle 사용하여 데이터를 무작위로 섞어줌

accrs = []

fold_idx = 1

for train_idx, test_idx in kf.split(final_data2): # 10번 반복
    print('fold {}'.format(fold_idx))
    
    train_d, test_d = final_data2.iloc[train_idx], final_data2.iloc[test_idx] # 학습 데이터, 훈련 데이터 나누기
    
    scaler = MinMaxScaler()
    train_d_scale = scaler.fit_transform(train_d[features])
    test_d_scale = scaler.transform(test_d[features])
    
    # PCA를 사용하였을 때 MSE가 더 올라감 
    #pca = PCA(n_components=10)

    train_y = train_d['user_rating']
    train_x = train_d_scale

    test_y = test_d['user_rating']
    test_x = test_d_scale

    model = LinearRegression()
    model.fit(train_x, train_y)
    
    # test_x에 대한 예측값 pred_y 구하기
    pred_y = model.predict(test_x)
    
    #모델 학습 후, 실제 값과 예측 값을 파라메터로 전달하여 MAE 메소드 호출
    mae = MAE(test_y, pred_y)
    
    accrs.append(mae)
    print("MAE : ", mae)
    
    fold_idx += 1

print("Total (Average) MAE(Mean absolute error) = ", np.average(accrs)) # 총 10번의 fold 평균 구하기

fold 1
MAE :  0.6134963525810563
fold 2
MAE :  0.6320801017644346
fold 3
MAE :  0.6167641098778264
fold 4
MAE :  0.6961810685006219
fold 5
MAE :  0.6197682309333679
fold 6
MAE :  0.6502202218297419
fold 7
MAE :  0.6104439174357214
fold 8
MAE :  0.615921929466631
fold 9
MAE :  0.6512758545999803
fold 10
MAE :  0.6380101928384624
Total (Average) MAE(Mean absolute error) =  0.6344161979827844


In [1]:
from sklearn.model_selection import train_test_split, KFold
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import numpy as np
import pandas as pd

# 평균 절대 오차(Mean absolute error): MAE, 낮을수록 좋음
def MAE(test_y, pred_y):
    return np.mean(np.abs((test_y - pred_y)))

# csv 파일 불러오기
file_path = '/home/u1043/machinelearning/practice/datasets_30069_39285_AppleStore.csv'
store_data = pd.read_csv(file_path)

# 데이터 전처리 
# 필요 없는 feature Drop
del(store_data['Unnamed: 0'])
del(store_data['id'])
del(store_data['track_name'])
del(store_data['currency'])
del(store_data['vpp_lic'])

# ver열의 데이터들의 .을 기준으로 첫번째 숫자만 저장하기
ver_split = store_data["ver"].str.split(".")
store_data["ver"] = ver_split.str.get(0)

# ver열의 데이터중 'iOV 2' 문자열이 있는 행은 삭제
contains_string_df = store_data['ver'].str.contains('iOV 2|v2|v3|1,2|V3|v1|Update 1')
store_data = store_data[~contains_string_df]

# category 데이터는 인코딩
prime_genre = preprocessing.LabelEncoder()
final_data1 = store_data.copy()
final_data1['prime_genre'] = prime_genre.fit_transform(final_data1['prime_genre'])

cont_rating = preprocessing.LabelEncoder()
final_data2 = final_data1.copy()
final_data2['cont_rating'] = prime_genre.fit_transform(final_data2['cont_rating'])


features = ['size_bytes', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'lang.num']


kf = KFold(n_splits=10, shuffle=True)

accrs = []
accrs1 = []

fold_idx = 1


# 학습 시키기
for train_idx, test_idx in kf.split(final_data2):
    print('fold {}'.format(fold_idx))
    
    # train, test 데이터 나누기
    train_d, test_d = final_data2.iloc[train_idx], final_data2.iloc[test_idx]
    
    ## 데이터 스케일링 (변수들의 단위 차이로 인해 숫자의 스케일이 크게 달라지는 경우)
    standardScaler = StandardScaler()
    
    train_data_standardScaled = standardScaler.fit_transform(train_d[features])
    test_data_standardScaled = standardScaler.transform(test_d[features])
    
    # PCA
    pca = PCA(n_components=9)

    train_y = train_d['user_rating']
    train_x = pca.fit_transform(train_data_standardScaled)

    test_y = test_d['user_rating']
    test_x =  pca.fit_transform(test_data_standardScaled)

    # LinearRegression 모델 학습
    model = LinearRegression()
    model.fit(train_x, train_y)
    
    # test_x에 대한 예측값 pred_y
    pred_y = model.predict(test_x)
    
    # MAE
    mean_accr = MAE(test_y, pred_y)
    
    accrs.append(mean_accr)
    print("MAE : ", mean_accr)
    
    fold_idx += 1

print("Total (Average) accuracy = ", np.average(accrs))

fold 1
MAE :  1.131539559626897
fold 2
MAE :  1.0930309642253557
fold 3
MAE :  1.4372313529317082
fold 4
MAE :  1.250447842106445
fold 5
MAE :  1.0167996506756134
fold 6
MAE :  1.0793007493659748
fold 7
MAE :  1.3215673184221028
fold 8
MAE :  0.9641243787733762
fold 9
MAE :  1.021130966070048
fold 10
MAE :  1.1498023925805054
Total (Average) accuracy =  1.1464975174778027


In [197]:
from sklearn.model_selection import train_test_split, KFold
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import numpy as np
import pandas as pd

# 평균 절대 오차(Mean absolute error): MAE, 낮을수록 좋음
def MAE(test_y, pred_y):
    return np.mean(np.abs((test_y - pred_y)))

# csv 파일 불러오기
file_path = '/home/u1043/machinelearning/practice/datasets_30069_39285_AppleStore.csv'
store_data = pd.read_csv(file_path)

# 데이터 전처리 
# 필요 없는 feature Drop
del(store_data['Unnamed: 0'])
del(store_data['id'])
del(store_data['track_name'])
del(store_data['currency'])
del(store_data['vpp_lic'])

# ver열의 데이터들의 .을 기준으로 첫번째 숫자만 저장하기
ver_split = store_data["ver"].str.split(".")
store_data["ver"] = ver_split.str.get(0)

# ver열의 데이터중 'iOV 2' 문자열이 있는 행은 삭제
contains_string_df = store_data['ver'].str.contains('iOV 2|v2|v3|1,2|V3|v1|Update 1')
store_data = store_data[~contains_string_df]

# category 데이터는 인코딩
prime_genre = preprocessing.LabelEncoder()
final_data1 = store_data.copy()
final_data1['prime_genre'] = prime_genre.fit_transform(final_data1['prime_genre'])

cont_rating = preprocessing.LabelEncoder()
final_data2 = final_data1.copy()
final_data2['cont_rating'] = prime_genre.fit_transform(final_data2['cont_rating'])


features = ['size_bytes', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'lang.num']

kf = KFold(n_splits=10, shuffle=True)

accrs = []
accrs1 = []

fold_idx = 1

######################## Linear Regression (not PCA) #########################
# 학습 시키기
for train_idx, test_idx in kf.split(final_data2):
    print('fold {}'.format(fold_idx))
    
    # train, test 데이터 나누기
    train_d, test_d = final_data2.iloc[train_idx], final_data2.iloc[test_idx]
    
    ## 데이터 스케일링 (변수들의 단위 차이로 인해 숫자의 스케일이 크게 달라지는 경우)
    standardScaler = StandardScaler()
    
    train_data_standardScaled = standardScaler.fit_transform(train_d[features])
    test_data_standardScaled = standardScaler.transform(test_d[features])
    
    # PCA
    #pca = PCA(n_components=10)

    train_y = train_d['user_rating']
    train_x = train_data_standardScaled

    test_y = test_d['user_rating']
    test_x = test_data_standardScaled

    # LinearRegression 모델 학습
    model = LinearRegression()
    model.fit(train_x, train_y)
    
    # test_x에 대한 예측값 pred_y
    pred_y = model.predict(test_x)
    
    # MAE
    mean_accr = MAE(test_y, pred_y)
    
    accrs.append(mean_accr)
    print("MAE : ", mean_accr)
    
    fold_idx += 1

print("Total (Average) accuracy = ", np.average(accrs))

fold 1
MAE :  0.633668519839351
fold 2
MAE :  0.6415222261456608
fold 3
MAE :  0.7015531629851828
fold 4
MAE :  0.6191295227839295
fold 5
MAE :  0.591158189033405
fold 6
MAE :  0.6390434540599211
fold 7
MAE :  0.611026144468245
fold 8
MAE :  0.6574313740771114
fold 9
MAE :  0.6184131590798856
fold 10
MAE :  0.6554490005259245
Total (Average) accuracy =  0.6368394752998616


In [196]:
from sklearn.model_selection import train_test_split, KFold
from sklearn.linear_model import Lasso
from sklearn import preprocessing
from sklearn.decomposition import PCA
import numpy as np
import pandas as pd

# 평균 절대 오차(Mean absolute error): MAE, 낮을수록 좋음
def MAE(test_y, pred_y):
    return np.mean(np.abs((test_y - pred_y)))

# csv 파일 불러오기
file_path = '/home/u1043/machinelearning/practice/datasets_30069_39285_AppleStore.csv'
store_data = pd.read_csv(file_path)

# 데이터 전처리 
# 필요 없는 feature Drop
del(store_data['Unnamed: 0'])
del(store_data['id'])
del(store_data['track_name'])
del(store_data['currency'])
del(store_data['vpp_lic'])

# ver열의 데이터들의 .을 기준으로 첫번째 숫자만 저장하기
ver_split = store_data["ver"].str.split(".")
store_data["ver"] = ver_split.str.get(0)

# ver열의 데이터중 'iOV 2' 문자열이 있는 행은 삭제
contains_string_df = store_data['ver'].str.contains('iOV 2|v2|v3|1,2|V3|v1|Update 1')
store_data = store_data[~contains_string_df]

# category 데이터는 인코딩
prime_genre = preprocessing.LabelEncoder()
final_data1 = store_data.copy()
final_data1['prime_genre'] = prime_genre.fit_transform(final_data1['prime_genre'])

cont_rating = preprocessing.LabelEncoder()
final_data2 = final_data1.copy()
final_data2['cont_rating'] = prime_genre.fit_transform(final_data2['cont_rating'])


features = ['size_bytes', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'lang.num']

kf = KFold(n_splits=10, shuffle=True)

accrs = []
accrs1 = []

fold_idx = 1

######################## Lasso Regression #########################

for train_idx, test_idx in kf.split(final_data2):
    print('fold {}'.format(fold_idx))
    
    # train, test 데이터 나누기
    train_d, test_d = final_data2.iloc[train_idx], final_data2.iloc[test_idx]
    
    ## 데이터 스케일링 (변수들의 단위 차이로 인해 숫자의 스케일이 크게 달라지는 경우)
    standardScaler = StandardScaler()
    
    train_data_standardScaled = standardScaler.fit_transform(train_d[features])
    test_data_standardScaled = standardScaler.transform(test_d[features])
    
    # PCA
    #pca = PCA(n_components=10)

    train_y = train_d['user_rating']
    train_x = train_data_standardScaled

    test_y = test_d['user_rating']
    test_x = test_data_standardScaled

    model = Lasso(alpha=0.05, normalize=True)
    model.fit(train_x, train_y)
    
    predict_x = model.predict(test_x)
    
    mean_accr = MAE(test_y, predict_x)
    accrs.append(mean_accr)
    print("test_MAE: ", mean_accr)
    
    fold_idx += 1

print("Total (Average) accuracy = ", np.average(accrs))

fold 1
test_MAE:  1.2044342885641013
fold 2
test_MAE:  1.0823132549285994
fold 3
test_MAE:  1.112049944861595
fold 4
test_MAE:  1.1623896963195923
fold 5
test_MAE:  1.142790734168399
fold 6
test_MAE:  1.119648941407115
fold 7
test_MAE:  1.1605931301631796
fold 8
test_MAE:  1.139477700653277
fold 9
test_MAE:  1.129374152233589
fold 10
test_MAE:  1.1036317377422904
Total (Average) accuracy =  1.1356703581041736


In [233]:
from sklearn.model_selection import train_test_split, KFold
from sklearn.linear_model import LinearRegression, Ridge
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
import numpy as np
import pandas as pd

# 평균 절대 오차(Mean absolute error): MAE, 낮을수록 좋음
def MAE(test_y, predict_x):
    return np.mean(np.abs((test_y - predict_x)))

# csv 파일 불러오기
file_path = '/home/u1043/machinelearning/practice/datasets_30069_39285_AppleStore.csv'
store_data = pd.read_csv(file_path)


## 데이터 전처리 

# 필요 없는 feature Drop
del(store_data['Unnamed: 0'])
del(store_data['id'])
del(store_data['track_name'])
del(store_data['currency'])
del(store_data['vpp_lic'])

# ver열의 데이터들의 .을 기준으로 첫번째 숫자만 저장하기
ver_split = store_data["ver"].str.split(".") #
store_data["ver"] = ver_split.str.get(0)

# ver열의 데이터중 'iOV 2' 문자열이 있는 행은 삭제
contains_string_df = store_data['ver'].str.contains('iOV 2|v2|v3|1,2|V3|v1|Update 1')
store_data = store_data[~contains_string_df]

# category 데이터는 인코딩
prime_genre = preprocessing.LabelEncoder()
final_data1 = store_data.copy()
final_data1['prime_genre'] = prime_genre.fit_transform(final_data1['prime_genre'])

cont_rating = preprocessing.LabelEncoder()
final_data2 = final_data1.copy()
final_data2['cont_rating'] = prime_genre.fit_transform(final_data2['cont_rating'])


features = ['size_bytes', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'lang.num']


kf = KFold(n_splits=10, shuffle=True)
accrs = []

fold_idx = 1

######################## Ridge Regression #########################
# 학습 시키기
for train_idx, test_idx in kf.split(final_data2):
    print('fold {}'.format(fold_idx))
    
    # train, test 데이터 나누기
    train_d, test_d = final_data2.iloc[train_idx], final_data2.iloc[test_idx]
      
    ## 데이터 스케일링 (변수들의 단위 차이로 인해 숫자의 스케일이 크게 달라지는 경우)
    scaler = MinMaxScaler()
    train_d_scale = scaler.fit_transform(train_d[features])
    test_d_scale = scaler.transform(test_d[features])
    
    # PCA
    #pca = PCA(n_components=10)

    train_y = train_d['user_rating']
    train_x = train_d_scale

    test_y = test_d['user_rating']
    test_x = test_d_scale

    # Ridge Regression 모델 학습
    # Linear Regression, Lasso Regression, Ridge Regression 세 모델을 학습시킨 결과 Ridge가 다른 두 모델에 비해 정확도가 높게 나옴
    model = Ridge(alpha=0.01, normalize=True)
    model.fit(train_x, train_y)
    
    # test_x에 대한 예측값 pred_y
    pred_y = model.predict(test_x)
    
    # MAE
    mean_accr = MAE(test_y, pred_y)
    
    accrs.append(mean_accr)
    print("MAE : ", mean_accr)
    
    fold_idx += 1

print("Total (Average) accuracy = ", np.average(accrs))

fold 1
MAE :  0.651807274502234
fold 2
MAE :  0.6882845136368357
fold 3
MAE :  0.631812530332557
fold 4
MAE :  0.586233898344766
fold 5
MAE :  0.5732688287094669
fold 6
MAE :  0.6837616883114085
fold 7
MAE :  0.6604479461395996
fold 8
MAE :  0.6079023564743958
fold 9
MAE :  0.6203779400531599
fold 10
MAE :  0.6838071114821951
Total (Average) accuracy =  0.6387704087986619
